The times taken to query the base galaxy Table using obsMetadata and instance Catalogs is much larger than the time to query the galaxy table on fatboy directly. This records timeit results for these queries

## Setup

In [1]:
from __future__ import division, print_function, absolute_import

In [2]:
import time
import os
import pandas as pd
import numpy as np

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import seaborn as sns
sns.set()

/usr/local/manual/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [4]:
from lsst.sims.catalogs.measures.instance import InstanceCatalog
import lsst.sims.catUtils.baseCatalogModels as bcm
from lsst.sims.catalogs.generation.db import CatalogDBObject, DBObject
from lsst.sims.catUtils.baseCatalogModels.GalaxyModels import GalaxyTileObj, GalaxyObj
from lsst.sims.utils import ObservationMetaData

/Users/rbiswas/src/LSST/sims_catalogs_generation/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/Users/rbiswas/src/LSST/sims_catalogs_generation/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')


## Observation Meta Data

In [ ]:
# Shifted to corner
observerMetaData_2 = ObservationMetaData(boundType='box',
                                  pointingRA=2.,
                                  pointingDec=2.,
                                  boundLength=1.)

In [ ]:
# Shifted to corner
observerMetaData_1 = ObservationMetaData(boundType='box',
                                  pointingRA=2.,
                                  pointingDec=2.,
                                  boundLength=2.)

In [ ]:
# Original Base
observerMetaData = ObservationMetaData(boundType='box',
                                  pointingRA=0.,
                                  pointingDec=0.,
                                  boundLength=2.)

In [ ]:
# Shifted to corner
observerMetaData_3 = ObservationMetaData(boundType='box',
                                  pointingRA=2.,
                                  pointingDec=2.,
                                  boundLength=0.5)

In [ ]:
observerMetaData.boundLength

In [ ]:
def queryObsMetaData(obsMetaData):
    ra, dec, size = obsMetaData.pointingRA, obsMetaData.pointingDec, obsMetaData.boundLength
    raSize , decSize = (size, size)
    coords = np.asarray([ra - size, ra + size, dec - size, dec + size])
    
    coordTrans = np.remainder(coords + size, np.repeat(4., 4)) - size
    coordWraps =  np.floor_divide(coords[[1,3]] - coords[[0,2]] , (4,4))
    querylims = coordTrans[2], coordTrans[0], coordTrans[3], coordTrans[1]
    
    
    if coordTrans[2] < coordTrans[0]:
        print('split')
    if coordTrans[3] < coordTrans[1]:
        print('split')
        
    #bounds = [Polygon(vert, fill=False, lw=2, alpha=1., ls='dashed') for vert in verts]
    return coords, coordTrans, coordWraps, querylims


In [ ]:
queryObsMetaData(observerMetaData)

In [ ]:
queryObsMetaData(observerMetaData_1)

In [ ]:
queryObsMetaData(observerMetaData_2)

In [ ]:
queryObsMetaData(observerMetaData_3)

In [ ]:
queryObsMetaData(observerMetaData_3)

In [ ]:
def tileObsMetaData(obsMetaData, num):
    ra, dec, size = obsMetaData.pointingRA, obsMetaData.pointingDec, obsMetaData.boundLength
    blen = size / num
    pras = np.arange(ra - size, ra + size, 2.0 *blen) + blen
    pdecs = np.arange(dec - size, dec + size, 2.0 * blen) + blen
    
    coords = np.dstack(np.meshgrid(pras, pdecs)).reshape(len(pras) * len(pdecs), 2)
    obsmdata = [ObservationMetaData(boundType='box',
                                  pointingRA=coords[i, 0],
                                  pointingDec=coords[i, 1],
                                  boundLength=blen) for i in range(len(coords))]
    # return coords
    diffvec = np.array([blen, blen])
    transvec = np.array([1, -1])
    verts = [[coord - diffvec, coord - transvec * diffvec, coord + diffvec, coord + diffvec *transvec] for coord in coords]  
    bounds = [Polygon(vert, fill=False, lw=2, alpha=1., ls='dashed') for vert in verts]
    return obsmdata, coords, bounds



In [ ]:
tiledObsMD, coords, bounds = tileObsMetaData(observerMetaData, 3)
fig, ax =plt.subplots()
ax.scatter(coords[:,0], coords[:, 1])
for bound in bounds:
    ax.add_patch(bound)
ax.set_xlim(-2., 2.)
ax.set_ylim(-2., 2.)

In [ ]:
len(tiledObsMD)

In [ ]:
class galCopy(InstanceCatalog):
    column_outputs = ['galtileid', 'raJ2000', 'decJ2000', 'redshift']#, 'a_d', 'b_d', 'pa_disk', 'mass_stellar', 'absmag_r_total']
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e', 'a_d': '%8e', 'b_d': '%8e', 'pa_disk': '%8e', 
                        'mass_stellar': '%8e', 'absmag_r_total': '%8e'}

In [ ]:
def writeCat(obsMD=tiledObsMD, redshiftmax=0.4, fname='gB-test', chunkSize=None):
    constr = 'redshift < {0}'.format(redshiftmax)
    galaxyTiled = GalaxyTileObj()
    print(len(obsMD))
    filenames = []
    for i, obsMetaData in enumerate(obsMD):
        filename = fname  + '{}'.format(i) + '.csv'
        filenames.append(filename)
        galaxyBase = galCopy(galaxyTiled, obs_metadata=obsMetaData, constraint=constr)
        galaxyBase.write_catalog(filename, chunkSize)
    return filenames

In [ ]:
# No need
def writeCatBase(redshiftmax=0.4, fname='gB-test.csv', chunkSize=None):
    constr = 'redshift < {0}'.format(redshiftmax)
    galaxyTiled = GalaxyObj()
    galaxyBase = galBaseCopy(galaxyTiled, obs_metadata=obsMetaData, constraint=constr)
    galaxyBase.write_catalog(fname, chunkSize)

In [ ]:
tileObsMD, coords, bounds = tileObsMetaData(observerMetaData, 1)

In [ ]:
len(tileObsMD)

In [ ]:
%%timeit
chunksize = None
tstart = time.time()
fnames = writeCat(obsMD=tileObsMD, chunkSize=chunksize)
print(fnames)
tend = time.time()
print(tend - tstart)
numlines = 0

In [ ]:
tileObsMD, coords, bounds = tileObsMetaData(observerMetaData, 2)

In [ ]:
%%timeit
chunksize = None
tstart = time.time()
fnames = writeCat(obsMD=tileObsMD, chunkSize=chunksize)
print(fnames)
tend = time.time()
print(tend - tstart)
numlines = 0

In [ ]:
tileObsMD, coords, bounds = tileObsMetaData(observerMetaData, 3)

In [ ]:
%%timeit
chunksize = None
tstart = time.time()
fnames = writeCat(obsMD=tileObsMD, chunkSize=chunksize)
print(fnames)
tend = time.time()
print(tend - tstart)
numlines = 0

In [ ]:
query = """EXECUTE [LSSTCATSIM].[dbo].[GalaxySearch2015l]
                   @ApertureStr = 'REGION RECT J2000 358.000000 -2.000000 2.000000 2.000000', @ColumnNames = 'dec as decJ2000,ra as raJ2000,redshift as redshift',
                   @ComponentSubset = 'ALL' , @WhereClause = 'redshift < 0.4’
        """


In [ ]:
numLines = 1
for i in range(9):
    fname = "gB-test{}.csv".format(i)
    with open(fname, 'r') as fp:
        numLines += sum(1 for line in fp) - 1 

In [ ]:
print(numLines)

In [ ]:
!tail gB-test0.csv

In [ ]:
!wc -l *.csv

In [ ]:
def writeDBObjBase(redshiftmax=0.4, fname='gB-test.csv', chunkSize=None):
    constr = 'redshift < {0}'.format(redshiftmax)
    galaxyBase = GalaxyTileObj()
    # galDB = dbobject = DBObject(database=config.database, driver='mssql+pymssql', host=config.host, port=config.port, connection=None)
    #galaxyBase = galBaseCopy(galaxyTiled, obs_metadata=obsMetaData, constraint=constr)
    query = """SELECT galtileid, raJ2000, decJ2000, redshift FROM galaxy WHERE redshift < 0.4 AND raJ2000 < 2 and raJ2000 > -2 and decJ2000 < 2 and decJ2000 > -2"""
    x = galaxyBase.execute_arbitrary(query)
    print(len(x), len(x[0]))
    #galaxyBase.write_catalog(fname, chunkSize)

In [ ]:
def writeDBObjBase(redshiftmax=0.4, fname='gB-test.csv', chunkSize=None):
    constr = 'redshift < {0}'.format(redshiftmax)
    galaxyBase = GalaxyObj()
    # galDB = dbobject = DBObject(database=config.database, driver='mssql+pymssql', host=config.host, port=config.port, connection=None)
    #galaxyBase = galBaseCopy(galaxyTiled, obs_metadata=obsMetaData, constraint=constr)
    query = """SELECT id, ra, dec, redshift WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
    x = galaxyBase.execute_arbitrary(query)
    print(len(x), len(x[0]))
    #galaxyBase.write_catalog(fname, chunkSize)

In [ ]:
%%timeit
chunksize = None
tstart = time.time()
writeDBObjBase(chunkSize=chunksize)
tend = time.time()
print(tend - tstart)

In [ ]:
%%timeit
chunksize = None
tstart = time.time()
writeCat(chunkSize=chunksize)
tend = time.time()
print(tend - tstart)

In [ ]:
%%timeit
chunksize = 50000
tstart = time.time()
writeCat(chunkSize=chunksize)
tend = time.time()
print(tend - tstart)

In [ ]:
%%timeit
chunksize = 500000
tstart = time.time()
writeCat(chunkSize=chunksize)
tend = time.time()
print(tend - tstart)

In [ ]:
%%timeit
chunksize = 5000
tstart = time.time()
writeCat(chunkSize=chunksize)
tend = time.time()
print(tend - tstart)

In [ ]:
%%timeit
chunksize = 50
tstart = time.time()
writeCat(chunkSize=chunksize)
tend = time.time()
print(tend - tstart)

In [ ]:
# No need
class galBaseCopy(InstanceCatalog):
    column_outputs = ['id', 'raJ2000', 'decJ2000', 'redshift']#, 'a_d', 'b_d', 'pa_disk', 'mass_stellar', 'absmag_r_total']
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e', 'a_d': '%8e', 'b_d': '%8e', 'pa_disk': '%8e', 
                        'mass_stellar': '%8e', 'absmag_r_total': '%8e'}

## Direct query with fetch

In [22]:
from lsst.utils import getPackageDir
from lsst.daf.persistence import DbAuth
import lsst.pex.config as pexConfig
import pymssql

In [18]:
def build_queries(obsMetaData):
    
    

IndentationError: expected an indented block (<ipython-input-18-2a622a949d29>, line 3)

In [19]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 1.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
x = db.fetchall()
print(len(x), len(x[0]))
tend = time.time()
print(tend - tstart)

7158432 4
24.9573810101
7158432 4
24.0226259232
7158432 4
23.8228468895


KeyboardInterrupt: 

In [ ]:
query = "SELECT "

In [30]:
query = """SELECT count(*) FROM galaxy WHERE ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)

In [32]:
13758768/ 1.0e6

13.758768

In [31]:
db.fetchall()

[(13758768,)]

In [26]:
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()

In [ ]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 1 and ra > 0 and dec < 1 and dec > 0"""
db.execute(query)
x = db.fetchall()
print(len(x), len(x[0]))
tend = time.time()
print((tend - tstart) * 16.)

In [ ]:
%%timeit

tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
x = db.fetchall()
print(len(x), len(x[0]))
tend = time.time()
print(tend - tstart)

In [25]:
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)



In [ ]:
%%timeit

tstart = time.time()
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 1.4 AND ra < 1 and ra > -0 and dec < 1 and dec > -0"""
db.execute(query)
x = db.fetchall()
print(len(x), len(x[0]))
tend = time.time()
print(16.0*(tend - tstart))

In [ ]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
x = db.fetchall()
print(len(x), len(x[0]))
tend = time.time()
print(tend - tstart)

In [ ]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """EXECUTE [LSSTCATSIM].[dbo].[GalaxySearch2015]
                   @ApertureStr = 'REGION RECT J2000 358.000000 -2.000000 2.000000 2.000000', @ColumnNames = 'dec as decJ2000,ra as raJ2000,redshift as redshift',
                   @ComponentSubset = 'ALL' , @WhereClause = 'redshift < 0.4'
"""
db.execute(query)
x = db.fetchall()
print(len(x), len(x[0]))
tend = time.time()
print(tend - tstart)

In [ ]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """[LSSTCATSIM].[dbo].[GalaxySearch2015]
                   @ApertureStr = 'REGION RECT J2000 358.000000 -2.000000 2.000000 2.000000', @ColumnNames = 'dec as decJ2000,ra as raJ2000,redshift as redshift',
                   @ComponentSubset = 'ALL' , @WhereClause = 'redshift < 0.4'"""

db.execute(query)
x = db.fetchall()
print(len(x), len(x[0]))
tend = time.time()
print(tend - tstart)

In [ ]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
num = 50000
x = db.fetchmany(num)
while len(x)>0:
    x = db.fetchmany(num)
# print(len(x), len(x[0]))
tend = time.time()
print(tend - tstart)

In [ ]:
!pwd

In [ ]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
#num = 50000
x = db.fetchall()
# print(len(x), len(x[0]))
tend = time.time()
print(tend - tstart)

In [ ]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
num = 50000
x = db.fetchmany(num)
while len(x) > 0:
    x = db.fetchmany(num)
# print(len(x), len(x[0]))
#df = pd.DataFrame.from_records(x, columns=['id', 'ra', 'dec', 'redshift'])
#df.to_csv('gb_pd.test.csv', index=False)
tend = time.time()
print(tend - tstart)

In [ ]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
#num = 50000
x = db.fetchall()
# print(len(x), len(x[0]))
df = pd.DataFrame.from_records(x, columns=['id', 'ra', 'dec', 'redshift'])
df.to_csv('gb_pd.test.csv', index=False)
tend = time.time()
print(tend - tstart)

In [ ]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT id, ra, dec, redshift FROM galaxy WHERE redshift < 0.4 AND ra < 2 and ra > -2 and dec < 2 and dec > -2"""
db.execute(query)
#num = 50000
x = db.fetchall()
# print(len(x), len(x[0]))
df = pd.DataFrame.from_records(x, columns=['id', 'ra', 'dec', 'redshift'])
df.to_csv('gb_pd.test.csv', index=False)
tend = time.time()
print(tend - tstart)

In [ ]:
%%timeit
tstart = time.time()
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)
db = DBConnection.cursor()
query = """SELECT galaxy.id AS id, ra*PI()/180. AS [raJ2000], dec*PI()/180. AS [decJ2000], galaxy.redshift AS redshift 
FROM galaxy  WITH(FORCESEEK) JOIN (SELECT htmidstart, htmidend 
FROM fHtmCoverRegion(%(fHtmCoverRegion_1)s)) AS anon_1 ON galaxy.htmid BETWEEN anon_1.htmidstart AND anon_1.htmidend 
WHERE sph.fRegionContainsXYZ(sph.fSimplifyString(%(fSimplifyString_1)s), galaxy.cx, galaxy.cy, galaxy.cz) = %(fRegionContainsXYZ_1)s
"""
db.execute(query)
#num = 50000
x = db.fetchall()
# print(len(x), len(x[0]))
df = pd.DataFrame.from_records(x, columns=['id', 'ra', 'dec', 'redshift'])
df.to_csv('gb_pd.test.csv', index=False)
tend = time.time()
print(tend - tstart)

In [ ]:
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))
DBConnection = pymssql.connect(user=DbAuth.username(config.host, config.port),
                               password=DbAuth.password(config.host, config.port),
                               database=config.database, port=config.port)



In [ ]:
config

In [ ]:
dbobject = DBObject(database=config.database, driver='mssql+pymssql', host=config.host, port=config.port, connection=None)

In [ ]:
dbobject.execute_arbitrary()

## Test Rotations

In [5]:
deltaAng =  np.radians(2)

In [6]:
coords = np.random.uniform((- deltaAng + np.pi/2., -deltaAng), (deltaAng + np.pi/2., deltaAng), size=(10e6, 2))

/usr/local/manual/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':


In [7]:
df = pd.DataFrame(coords, columns=['theta', 'phi'])

In [8]:
df.describe()

,theta,phi
count,10000000.000000,10000000.000000
mean,1.570779,-0.000003
std,0.020154,0.020150
min,1.535890,-0.034907
25%,1.553319,-0.017445
50%,1.570762,-0.000008
75%,1.588234,0.017444
max,1.605703,0.034907


In [9]:
def rotation(df, delta=0):
    """
    delta : int
    """
    d = delta
    delta = np.radians(delta * 4.0)
    cosDelta = np.cos(delta)
    sinDelta = np.sin(delta)
    
    sintheta = np.sin(df.theta)
    costheta = np.cos(df.theta)
    cosphi = np.cos(df.phi)
    # sinphi = np.sin(df.phi)
    
    zprime =  costheta * cosDelta - sinDelta * sintheta * cosphi
    xprime = sintheta * cosphi * cosDelta + costheta * sinDelta
    # yprime = sintheta * sinphi
    
    #rlat = np.sqrt(xprime **2. + yprime **2.)
    rlat = np.sqrt(1.0 - zprime**2)
    #print(min(rlat), max(rlat))
    thetaprime = 'thetaprime_{}'.format(int(d))
    phiprime = 'phiprime_{}'.format(d)
    df[phiprime] = np.arccos(xprime / rlat)
    #df['x'] = sintheta * cosphi
    #df['y'] = sintheta * sinphi
    #df['z'] = costheta
    #df['xprime'] = xprime
    #df['yprime'] = yprime
    #df['zprime'] = zprime
    df[thetaprime] = np.arccos(zprime)
    #df[thetaprime] = np.arctan(rlat/zprime) 
    #df.thetaprime[df[thetaprime] < 0 ] += np.pi
    #df[thetaprime] = np.arcsin(np.sqrt(xprime**2. + yprime**2))#*np.sign(zprime)
    # df[thetaprime] = np.arcsin(rlat) * np.sign(zprime)

In [10]:
%timeit rotated(df, delta=0)

NameError: global name 'rotated' is not defined

In [13]:
%timeit rotated(df, delta=1)

1 loops, best of 3: 1.73 s per loop


In [14]:
%timeit rotated(df, delta=4)

1 loops, best of 3: 1.65 s per loop


In [15]:
%timeit rotation(df, delta=1)

1 loops, best of 3: 1.14 s per loop


In [16]:
%timeit rotation(df, delta=4)

1 loops, best of 3: 1.13 s per loop


In [12]:
def rotated(df, delta=0):
    """
    delta : int
    """
    d = delta
    delta = np.radians(delta * 4.0)
    cosDelta = np.cos(delta)
    sinDelta = np.sin(delta)
    
    sintheta = np.sin(df.theta)
    costheta = np.cos(df.theta)
    cosphi = np.cos(df.phi)
    sinphi = np.sin(df.phi)
    
    zprime =  costheta * cosDelta - sinDelta * sintheta * cosphi
    xprime = sintheta * cosphi * cosDelta + costheta * sinDelta
    yprime = sintheta * sinphi
    
    rlat = np.sqrt(xprime **2. + yprime **2.)
    #print(min(rlat), max(rlat))
    thetaprime = 'thetaprime_{}'.format(int(d))
    phiprime = 'phiprime_{}'.format(d)
    df[phiprime] = np.arcsin(yprime / rlat)
    df['x'] = sintheta * cosphi
    df['y'] = sintheta * sinphi
    df['z'] = costheta
    df['xprime'] = xprime
    df['yprime'] = yprime
    df['zprime'] = zprime
    df[thetaprime] = np.arccos(zprime)
    #df[thetaprime] = np.arctan(rlat/zprime) 
    #df.thetaprime[df[thetaprime] < 0 ] += np.pi
    #df[thetaprime] = np.arcsin(np.sqrt(xprime**2. + yprime**2))#*np.sign(zprime)
    # df[thetaprime] = np.arcsin(rlat) * np.sign(zprime)

In [ ]:
def rotated(df, delta=0):
    """
    delta : int
    """
    d = delta
    delta = np.radians(delta * 4.0)
    cosDelta = np.cos(delta)
    sinDelta = np.sin(delta)
    
    sintheta = np.sin(df.theta)
    costheta = np.cos(df.theta)
    cosphi = np.cos(df.phi)
    sinphi = np.sin(df.phi)
    
    zprime =  costheta * cosDelta - sinDelta * sintheta * cosphi
    xprime = sintheta * cosphi * cosDelta + costheta * sinDelta
    yprime = sintheta * sinphi
    
    rlat = np.sqrt(xprime **2. + yprime **2.)
    #print(min(rlat), max(rlat))
    thetaprime = 'thetaprime_{}'.format(int(d))
    phiprime = 'phiprime_{}'.format(d)
    df[phiprime] = np.arcsin(yprime / rlat)
    df['x'] = sintheta * cosphi
    df['y'] = sintheta * sinphi
    df['z'] = costheta
    df['xprime'] = xprime
    df['yprime'] = yprime
    df['zprime'] = zprime
    df[thetaprime] = np.arccos(zprime)
    #df[thetaprime] = np.arctan(rlat/zprime) 
    #df.thetaprime[df[thetaprime] < 0 ] += np.pi
    #df[thetaprime] = np.arcsin(np.sqrt(xprime**2. + yprime**2))#*np.sign(zprime)
    # df[thetaprime] = np.arcsin(rlat) * np.sign(zprime)

In [ ]:
#def rotated(df, delta=0):
    """
    delta : int
    """
    d = delta
    delta = np.radians(delta * 4.0)
    cosDelta = np.cos(delta)
    sinDelta = np.sin(delta)
    
    sintheta = np.sin(df.theta)
    costheta = np.cos(df.theta)
    cosphi = np.cos(df.phi)
    
    costhetaprime =  costheta * cosDelta - sinDelta * sintheta * cosphi
    sinthetaprime = np.sqrt(1.0 - costhetaprime**2.) * np.sign(df.theta)
    cosphiprime = (sintheta * cosphi * cosDelta + costheta * sinDelta ) /sinthetaprime
    df['costhetprime']= costhetaprime
    df['cosphiprime'] = cosphiprime
    df['thetaprime'] = np.arcsin(sinthetaprime)
    df['phiprime'] = np.arcsin(cosphiprime) * np.sign(df.phi)
    

In [ ]:
_ = np.arcsin(np.arange(-1., 1., 0.01))

In [ ]:
ratio = lambda x: (np.arccos(x) - np.arccos(0.) ) / np.arcsin(x)

In [ ]:
ratio(1.0e-5)

In [ ]:
np.arccos(np.arange(-0., 1., 0.001))

In [ ]:
%timeit rotated(df, delta=2)

In [ ]:
#rotated(df, delta=deltaAng*2.*3)

In [ ]:
np.allclose(df.theta, df.thetaprime_0)

In [ ]:
np.allclose(df.phi, df.phiprime_0)

In [ ]:
plt.hist(df.phi)

In [ ]:
df.head()

In [ ]:
np.arctan(np.arange(-100., 100., 0.01)) + np.pi/2.

In [ ]:
def angdist(df):
    df['cosine'] = df['x'] *df['xprime'] + df['y'] *df['yprime'] + df['z'] * df['zprime']

In [ ]:
df.dot()

In [ ]:
angdist(df)

In [ ]:
_ = plt.plot(df.phi, df.cosine.apply(np.arccos).apply(np.degrees) - 4.0, '.')

In [ ]:
np.allclose(df.z, df.zprime)

In [ ]:
np.arccos(df.z)

In [ ]:
df_a = df.ix[np.random.random_integers(0, len(df), size=10000)]
df_b = df.ix[np.random.random_integers(0, len(df), size=10000)]

In [ ]:
df_a.head()

In [ ]:
df.ix[1]

In [ ]:
def angSep( df1, df2, theta, phi):
    th1 = df1[theta].values
    th2 = df2[theta].values
    ph1 = df1[phi].values
    ph2 = df2[phi].values

    cos = np.sin(th1)* np.sin(th2)* np.cos(ph1- ph2) + np.cos(th1)*np.cos(th2)
    return cos

In [ ]:
plt.plot(df_a['theta'] , 
         angSep(df_a, df_b, 'theta', 'phi')- angSep(df_a, df_b, 'thetaprime_2', 'phiprime_2'),'.')

In [ ]:
x = angSep(df_a, df_b, 'theta', 'phi')- angSep(df_a, df_b, 'thetaprime', 'phiprime')

In [ ]:
x

In [ ]:
np.max(x)

In [ ]:
len(x[np.isnan(x)])

In [ ]:
(df.thetaprime - df.theta).apply(np.degrees).hist()

In [ ]:
df.theta.describe()

In [ ]:
plt.plot(df.theta.astype(np.float).values, df.thetaprime.astype(np.float).values - df.theta.astype(np.float).values, '.')

In [ ]:
len(df.theta)/1.0e6

In [ ]:
x = rotated(df, delta=2*deltaAng) - df.theta.values

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.hist(np.degrees(x))

In [ ]:
!pwd

##  Transformations

In [ ]:
def rotated(df, delta=0):
    """
    delta : int
    """
    d = delta
    delta = np.radians(delta * 4.0)
    cosDelta = np.cos(delta)
    sinDelta = np.sin(delta)
    
    sintheta = np.sin(df.theta)
    costheta = np.cos(df.theta)
    cosphi = np.cos(df.phi)
    sinphi = np.sin(df.phi)
    
    zprime =  costheta * cosDelta - sinDelta * sintheta * cosphi
    xprime = sintheta * cosphi * cosDelta + costheta * sinDelta
    yprime = sintheta * sinphi
    
    rlat = np.sqrt(xprime **2. + yprime **2.)
    #print(min(rlat), max(rlat))
    thetaprime = 'thetaprime_{}'.format(int(d))
    phiprime = 'phiprime_{}'.format(d)
    df[phiprime] = np.arcsin(yprime / rlat)
    df['x'] = sintheta * cosphi
    df['y'] = sintheta * sinphi
    df['z'] = costheta
    df['xprime'] = xprime
    df['yprime'] = yprime
    df['zprime'] = zprime
    df[thetaprime] = np.arccos(zprime)
    #df[thetaprime] = np.arctan(rlat/zprime) 
    #df.thetaprime[df[thetaprime] < 0 ] += np.pi
    #df[thetaprime] = np.arcsin(np.sqrt(xprime**2. + yprime**2))#*np.sign(zprime)
    # df[thetaprime] = np.arcsin(rlat) * np.sign(zprime)

In [ ]:
def angSep( df1, df2, theta, phi):
    th1 = df1[theta].values
    th2 = df2[theta].values
    ph1 = df1[phi].values
    ph2 = df2[phi].values

    cos = np.sin(th1)* np.sin(th2)* np.cos(ph1- ph2) + np.cos(th1)*np.cos(th2)
    return cos

###  Tests of rotation

In [ ]:
deltaAng =  np.radians(2)

In [ ]:
coords = np.random.uniform((- deltaAng + np.pi/2., -deltaAng), (deltaAng + np.pi/2., deltaAng), size=(10e6, 2))

In [ ]:
df = pd.DataFrame(coords, columns=['theta', 'phi'])

In [ ]:
def rotation(df, delta=0):
    """
    delta : int
    """
    d = delta
    delta = np.radians(delta * 4.0)
    cosDelta = np.cos(delta)
    sinDelta = np.sin(delta)
    
    df['theta'] = - df['decJ2000'] + np.pi/2.
    df['phi'] = df['raJ2000'] - np.radians(2.)
    sintheta = np.sin(df.theta)
    costheta = np.cos(df.theta)
    cosphi = np.cos(df.phi)
    # sinphi = np.sin(df.phi)
    
    zprime =  costheta * cosDelta - sinDelta * sintheta * cosphi
    xprime = sintheta * cosphi * cosDelta + costheta * sinDelta
    # yprime = sintheta * sinphi
    
    #rlat = np.sqrt(xprime **2. + yprime **2.)
    rlat = np.sqrt(1.0 - zprime**2)
    #print(min(rlat), max(rlat))
    thetaprime = 'thetaprime_{}'.format(int(d))
    phiprime = 'phiprime_{}'.format(d)
    df[phiprime] = np.arccos(xprime / rlat) - np.radians(2.)
    #df['x'] = sintheta * cosphi
    #df['y'] = sintheta * sinphi
    #df['z'] = costheta
    #df['xprime'] = xprime
    #df['yprime'] = yprime
    #df['zprime'] = zprime
    df[thetaprime] = np.arccos(zprime)
    #df[thetaprime] = np.arctan(rlat/zprime) 
    #df.thetaprime[df[thetaprime] < 0 ] += np.pi
    #df[thetaprime] = np.arcsin(np.sqrt(xprime**2. + yprime**2))#*np.sign(zprime)
    # df[thetaprime] = np.arcsin(rlat) * np.sign(zprime)

In [ ]:
%timeit rotated(df, delta=1)

In [ ]:
%timeit rotation(df, delta=1)

In [ ]:
df.head()

In [ ]:
plt.plot(df.theta, (df.thetaprime_1 - df.theta), '.')

In [ ]:
plt.plot(df.phi, (df.thetaprime_1 - df.theta - np.radians(4.)), '.')

In [ ]:
plt.plot(df.phi, (df.phiprime_1 - df.phi), '.')

In [ ]:
plt.plot(df.theta, (df.thetaprime_1 - df.theta - np.radians(4.)), '.')

Select at random 2n points from df and find their angular separations before and after the transformations

In [ ]:
Nchecks = 1000
sel = np.random.choice(df.index, 2*Nchecks, replace=False)

In [ ]:
np.unique(sel).size == sel.size

In [ ]:
len(sel)

In [ ]:
df_a = df.ix[sel[:len(sel)/2.]]
df_b = df.ix[sel[len(sel)/2:]]

In [ ]:
print(map(len, (df_a, df_b)))

In [ ]:
SepsBefore = angSep(df_a, df_b, 'theta', 'phi')

In [ ]:
SepsAfter = angSep(df_a, df_b, 'thetaprime_1', 'phiprime_1')

In [ ]:
np.allclose(SepsAfter, SepsBefore)

## Catsim Queries

In [ ]:
observerMetaData = ObservationMetaData(boundType='box',
                                  pointingRA=0.,
                                  pointingDec=0.,
                                  boundLength=0.1)

In [ ]:
class galCopy(InstanceCatalog):
    column_outputs = ['galtileid', 'raJ2000', 'decJ2000', 'redshift']#, 'a_d', 'b_d', 'pa_disk', 'mass_stellar', 'absmag_r_total']
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e', 'a_d': '%8e', 'b_d': '%8e', 'pa_disk': '%8e', 
                        'mass_stellar': '%8e', 'absmag_r_total': '%8e'}

In [ ]:
def galTileID(tileID, galID):
    return 100000000* tileID + galID

def galIds(tileID, galTileID):
    return galTileID - 100000000* tileID

In [ ]:
galaxyTiled = GalaxyTileObj()

In [ ]:
constr='redshift < 0.2'
gBase = galCopy(galaxyTiled, obs_metadata=observerMetaData, constraint=constr)
gBase.write_catalog('gBase.csv')
gBase_df  = pd.read_csv('gBase.csv', delimiter=', ',index_col='#galtileid')
gBase_df.index.name = 'galtileid'
gBase_df.head()

In [ ]:
gBase_df['id'] = galIds(22, gBase_df.index.values)
gBase_df.set_index('id', inplace=True)
gBase_df.head()

In [ ]:
obsMetaData_1 = observerMetaData

In [ ]:
obsMetaData_1.pointingDec = -4.0

In [ ]:
obsMetaData_1.boundLength

In [ ]:
constr='redshift < 0.2'
gTile_1 = galCopy(galaxyTiled, obs_metadata=observerMetaData, constraint=constr)

In [ ]:
gTile_1.write_catalog('gTile_1.csv')

In [ ]:
gTile1_df  = pd.read_csv('gTile_1.csv', delimiter=', ', index_col='#galtileid')
gTile1_df.index.name = 'galtileid'
gTile1_df.head()

In [ ]:
gTile1_df['id'] = galIds(21, gTile1_df.index.values)

In [ ]:
gTile1_df.set_index('id', inplace=True)

In [ ]:
gTile1_df.head()

In [ ]:
gTile1_df.index.values.min()

In [ ]:
gTile1_df.rename(columns={'raJ2000':'raJ2000_1', 'decJ2000':'decJ2000_1', 'redshift': 'redshift_1'},
                 inplace=True)

In [ ]:
merged = pd.concat([gBase_df, gTile1_df], axis=1)

In [ ]:
merged['shiftedRA']  = np.where(merged.raJ2000 < 3., merged.raJ2000, merged.raJ2000 - 2.0*np.pi)

In [ ]:
merged['shiftedRA_1']  = np.where(merged.raJ2000_1 < 3., merged.raJ2000_1, merged.raJ2000_1 - 2.0*np.pi)

In [ ]:
merged.head()

In [ ]:
uncommon = merged[merged.isnull().any(axis=1)].copy()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(6, 12), sharex=True)
ax[0].plot(merged.shiftedRA, merged.decJ2000, 'y.')
ax[0].scatter(uncommon.shiftedRA, uncommon.decJ2000, color='b')

ax[1].plot(merged.shiftedRA_1, merged.decJ2000_1, 'y.')
ax[1].scatter(uncommon.shiftedRA_1, uncommon.decJ2000_1, color='r')




In [ ]:
gTile1_df.sort('redshift').head()

In [ ]:
class galBaseCopy(InstanceCatalog):
    column_outputs = ['id', 'raJ2000', 'decJ2000', 'redshift']#, 'a_d', 'b_d', 'pa_disk', 'mass_stellar', 'absmag_r_total']
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e', 'a_d': '%8e', 'b_d': '%8e', 'pa_disk': '%8e', 
                        'mass_stellar': '%8e', 'absmag_r_total': '%8e'}

In [ ]:
galaxybaseObs = GalaxyObj()
galaxyBase = galBaseCopy(galaxybaseObs, obs_metadata=observerMetaData, constraint=constr)
galaxyBase.write_catalog('galaxyBase_small.csv')

In [ ]:
galBase_df = pd.read_csv('galaxyBase_small.csv', delimiter=', ', index_col='#id')
galBase_df.index.name = 'id'

In [ ]:
galBase_df.sort('redshift').head()

In [ ]:
from lsst.sims.utils import haversine

In [ ]:
orig_1 = gBase_df.ix[2207460316]
orig_2 = gBase_df.ix[2207460340]

In [ ]:
orig_2

In [ ]:
rot_1 = gTile1_df.ix[2107460316]
rot_2 = gTile1_df.ix[2107460340]

In [ ]:
rot_2

In [ ]:
orig_sep = haversine(orig_1.decJ2000, orig_1.raJ2000, orig_2.decJ2000, orig_2.raJ2000)

In [ ]:
rotated_sep = haversine(rot_1.decJ2000, rot_1.raJ2000, rot_2.decJ2000, rot_2.raJ2000)

In [ ]:
np.allclose(rotated_sep, orig_sep)

In [ ]:
def galTileID(tileID, galID):
    return 100000000* tileID + galID

def galIds(tileID, galTileID):
    return galTileID - 100000000* tileID

In [ ]:
gBase_df['Id'] = galIds(22, gBase_df.index.values)
gBase_df.set_index('Id', inplace=True)

In [ ]:
gBase_df.head()

In [ ]:
gTile1_df['Id'] = galIds(21, gTile1_df.index.values)

In [ ]:
gTile1_df.set_index('Id', inplace=True)

In [ ]:
gTile1_df.head()

In [ ]:
merged = pd.concat([gBase_df, gTile1_df], axis=1)

In [ ]:
merged

In [ ]:
galtileIDs = galTileID(21, galIdFromBase)

In [ ]:
galTileID_1 = gTile1_df.index.values
galTileID_1.sort()

In [ ]:
galTileID_1[:10]

In [ ]:
print(len(galtileIDs), len(galTileID_1))

In [ ]:
len(set(galtileIDs) - set(galTileID_1))

In [ ]:
len(set(galTileID_1) - set(galtileIDs))

In [ ]:
y = gBase_df.index.values
y.sort()

In [ ]:
y[:10]

In [ ]:
galtileIDs[:10]